In [75]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999


import time
import unicodedata

import datetime
from datetime import timedelta, datetime

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import glob

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

import locale
locale.setlocale(locale.LC_ALL, 'es_ES')

'es_ES'

# Import data

In [77]:
period = '20180210-3'

In [78]:
df = pd.read_csv('../data/output/scraped_data_clean_first_{}.csv'.format(period), encoding='utf-8', low_memory=False)

In [79]:
df.shape

(1954998, 23)

# Shifted table because of extra column 

# Contract filpped with Department 

In [80]:
df.loc[df['department'].str.contains('per')].head(1)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
174046,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,1,profesional,aedo,torres,BORIS CRISTIAN,,ADMINISTRACION PUBLICA,PROFESIONAL,5,1,Pesos,975.647,NO,15/01/2017,31/12/2017,PAGO PROPORCIONAL A LOS DÍAS TRABAJADOS EN EL...,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv


In [81]:
a = df.loc[df['department'].str.contains('per')].shape
b = round((a[0] / df.shape[0])*100,1)
print('{:,} rows of data with flipped Department and Contract ({}%)'.format(a[0], b))

178,466 rows of data with flipped Department and Contract (9.1%)


In [82]:
df['auxDept'] = [dept if 'per' not in dept else cont for (dept,cont) in zip(df['department'], df['contract'])]
df['auxCont'] = [cont if 'per' in cont else dept for (dept,cont) in zip(df['department'], df['contract'])]

df['department'] = df['auxDept']
df['contract'] = df['auxCont']

del df['auxCont']
del df['auxDept']

In [83]:
a = df.loc[df['department'].str.contains('per')].shape
b = round((a[0] / df.shape[0])*100,1)
print('{:,} rows of data with flipped Department and Contract ({}%)'.format(a[0], b))

1,023 rows of data with flipped Department and Contract (0.1%)


In [84]:
df.loc[df['department'].str.contains('per')].head(1)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
1305764,Ministerio del Trabajo y Previsión Social,Superintendencia de Seguridad Social (SUSESO),Superintendencia de Seguridad Social (SUSESO),Año 2008,allyear,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,http://163.247.55.110/PortalWEB/GTransparente/...,-1,-1,-1,-1,-1,-1,../data/output/scraped_data_20180210-3.csv


What's left could be dummy data (0.1% of rows)

# Salary into int 

## Cleaning script 

In [85]:
df['salary1'] = df['salary'].str.replace('.', '')

clean_list = ['.', ',', ' ', '\t', '$', '-', 'Pesos']
for c in clean_list:
    df['salary1'] = df['salary1'].str.replace(c, '')

clean_vals = ['', '0', '1', '10']
for c in clean_vals:
    df.loc[df['salary1'] == c] = None


## Checks 

In [86]:
df['salary1'].sort_values().head()

1740429     100000
724037     1000000
723844     1000000
724231     1000000
728733     1000000
Name: salary1, dtype: object

In [87]:
df['salary1'].sort_values().tail()

1954993    None
1954994    None
1954995    None
1954996    None
1954997    None
Name: salary1, dtype: object

## Results 

In [88]:
df['salary1'] = pd.to_numeric(df['salary1'], errors='coerce')

In [89]:
pd.isnull(df['salary1']).value_counts() / df.shape[0]

False    0.821408
True     0.178592
Name: salary1, dtype: float64

# Print output 

In [90]:
df.to_csv('../data/output/scraped_data_clean_second_{}.csv'.format(period), encoding='utf-8', index=False)